# data_augmentation

### Imports

In [ ]:
import os
from PIL import Image, ImageEnhance, ImageOps
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
import random
from itertools import product
import skimage.io
from skimage import color
from skimage import io
import glob
import cv2
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageEnhance

import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


### Functions

In [ ]:
# Función para añadir ruido a la imagen
def add_noise(image):
    np_image = np.array(image)
    ruido = np.random.normal(10, 25, np_image.shape).astype(np.int32)  # Nivel de ruido
    noisy_image = np.clip(np_image + ruido, 0, 255).astype(np.uint8)  # Asegurar valores válidos
    return Image.fromarray(noisy_image)

# Función para deformación elástica
def elastic_transform(image):
    """Elastic deformation of images as described in [Simard2003]_ (with modifications).
    """
    image = np.array(image)
    alpha = image.shape[1] * 2
    sigma = image.shape[1] * 0.08
    alpha_affine = image.shape[1] * 0.08

    shape = image.shape
    shape_size = shape[:2]
    random_state = np.random.RandomState(None)

    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size,
                       [center_square[0]+square_size, center_square[1]-square_size],
                       center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    # Elastic deformation (Gaussian noise)
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    # Apply elastic deformation
    deformed_image = map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

    # Convert back to PIL Image
    deformed_image_pil = Image.fromarray(deformed_image.astype(np.uint8))

    return deformed_image_pil

# Aplicar todas las transformaciones juntas
def augment_image(image):
    augmented_image = image

    # 1. Volteo horizontal
    augmented_image = ImageOps.mirror(augmented_image)

    # 2. Volteo vertical
    augmented_image = ImageOps.flip(augmented_image)

    # 3. Ajuste de brillo (ajuste más sutil)
    enhancer = ImageEnhance.Brightness(augmented_image)
    brightness_factor = random.uniform(0.5, 1.5)  # Antes: (0.6, 1.8)
    augmented_image = enhancer.enhance(0.5)

    # 4. Ajuste de contraste (ajuste más sutil)
    enhancer = ImageEnhance.Contrast(augmented_image)
    contrast_factor = random.uniform(0.5, 1.5)  # Antes: (0.6, 1.8)
    augmented_image = enhancer.enhance(contrast_factor)

     # 5. Cambio de saturación (ajuste más sutil)
    enhancer = ImageEnhance.Color(augmented_image)
    saturation_factor = random.uniform(0.5, 1.5)  # Ajuste sutil
    augmented_image = enhancer.enhance(saturation_factor)

    # 6. Añadir ruido (menos probabilidad de aplicarlo)
    #if random.random() < 0.5:  # 50% de probabilidad
    augmented_image = add_noise(augmented_image)

    # 7. Aplicar deformación elástica (probabilidad de aplicarla)
    #if random.random() < 0.:  # 30% de probabilidad
    augmented_image = elastic_transform(augmented_image)

    return augmented_image

def generar_nombre(imagen_original):
    nombre, ext = os.path.splitext(imagen_original)
    return f"{nombre}_aug{ext}"

### Augmentation

In [ ]:
# Directorio del dataset
dataset_path = "/content/drive/My Drive/Investigacion/UTN/GIAR/Dataset/clustering/AutoEncoder_agglomerative_v0/train/not"
save_path = dataset_path

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"El directorio {dataset_path} no existe.")

imagenes_originales = [
    file_name for file_name in os.listdir(dataset_path)
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg'))
]

# Determinar la cantidad máxima de imágenes aumentadas
cantidad_aumentadas = int(len(imagenes_originales) * 1)  # Hasta el 70%
contador = 0


for file_name in random.sample(imagenes_originales, cantidad_aumentadas):  # Selección aleatoria
    file_path = os.path.join(dataset_path, file_name)

    with Image.open(file_path) as img:
        # Aplicar augmentations
        augmented_image = augment_image(img)

        # Guardar la imagen
        nuevo_nombre = generar_nombre(file_name)
        nuevo_path = os.path.join(save_path, nuevo_nombre)
        augmented_image.save(nuevo_path)
        contador += 1

print(f"Se han generado {contador} imágenes aumentadas en {save_path}.")